<a href="https://colab.research.google.com/github/NajimovOtabek/Assignment_SW/blob/main/demo_transformer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1-t3RdDpmqMs4ABt9oobSapeNYTZJ9tpu
!unzip MachineLearningCSV.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-t3RdDpmqMs4ABt9oobSapeNYTZJ9tpu
To: /content/MachineLearningCSV.zip
100% 235M/235M [00:01<00:00, 172MB/s]
Archive:  MachineLearningCSV.zip
   creating: MachineLearningCVE/
  inflating: MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv  
  in

In [ ]:
!wget https://raw.githubusercontent.com/Jumabek/net_intrusion_detection/develop/preprocessing.py

--2023-08-25 01:26:59--  https://raw.githubusercontent.com/Jumabek/net_intrusion_detection/develop/preprocessing.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3790 (3.7K) [text/plain]
Saving to: ‘preprocessing.py’

preprocessing.py    100%[===================>]   3.70K  --.-KB/s    in 0s      

2023-08-25 01:27:00 (53.8 MB/s) - ‘preprocessing.py’ saved [3790/3790]



In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score, classification_report
from preprocessing import read_data, load_data, normalize
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import os

In [ ]:
dataroot = 'MachineLearningCVE/'
from preprocessing import read_data
data = read_data(dataroot,'*.pcap_ISCX.csv')

[########################################] | 100% Completed | 29.57 s


In [ ]:
# Load and normalize the data
X, y = load_data(dataroot)
X = normalize(X)

[########################################] | 100% Completed | 29.19 s
there are 2830743 flow records with 79 feature dimension
stripped column names
dropped bad columns
There are 0 nan entries
converted to numeric


In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import numpy as np

def balance_data(X, y, target_samples=50000, seed=42):
    # First, apply under-sampling to reduce the majority class
    under_sampler = RandomUnderSampler(sampling_strategy={0: target_samples}, random_state=seed)
    X_under, y_under = under_sampler.fit_resample(X, y)

    # Next, apply SMOTE to increase the minority classes
    smote = SMOTE(random_state=seed)
    new_X, new_y = smote.fit_resample(X_under, y_under)

    return new_X, new_y

In [ ]:
# Splitting the Data
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42)

# Print class distribution before balancing
unique_before, counts_before = np.unique(y, return_counts=True)
print("Class distribution before balancing:", dict(zip(unique_before, counts_before)))

# Balance the data
X_train, y_train = balance_data(X_train, y_train, target_samples=50000, seed=42)

# Print class distribution after balancing
unique_after, counts_after = np.unique(y_train, return_counts=True)
print("Class distribution after balancing:", dict(zip(unique_after, counts_after)))


# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

Class distribution before balancing: {0: 2273097, 1: 1966, 2: 128027, 3: 10293, 4: 231073, 5: 5499, 6: 5796, 7: 7938, 10: 158930, 11: 5897, 12: 1507, 14: 652}


/usr/local/lib/python3.10/dist-packages/imblearn/utils/_validation.py:313: UserWarning: After over-sampling, the number of samples (50000) in class 0 will be larger than the number of samples in the majority class (class #0 -> 484)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/imblearn/utils/_validation.py:313: UserWarning: After over-sampling, the number of samples (50000) in class 1 will be larger than the number of samples in the majority class (class #0 -> 484)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/imblearn/utils/_validation.py:313: UserWarning: After over-sampling, the number of samples (50000) in class 2 will be larger than the number of samples in the majority class (class #0 -> 484)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/imblearn/utils/_validation.py:313: UserWarning: After over-sampling, the number of samples (50000) in class 3 will be larger than the number of samples in the majority class (class #0 -> 484)
  warnings.warn(
/usr

Class distribution after balancing: {0: 50000, 1: 50000, 2: 50000, 3: 50000, 4: 50000, 5: 50000, 6: 50000, 7: 50000, 10: 50000, 11: 50000, 12: 50000, 14: 50000}


In [ ]:
from sklearn.metrics import confusion_matrix

class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=64, nhead=4, num_layers=4, dim_feedforward=128, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(1)  # Add a fake batch dimension for the transformer
        x = self.transformer(x, x)  # Encoder-Decoder Self-Attention
        x = x.squeeze(1)
        x = self.fc(x)
        return x



In [ ]:
from sklearn.metrics import balanced_accuracy_score, classification_report

def evaluate_model(model, data_loader) -> tuple:
    model.eval()
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for data, labels in data_loader:
            data = data.cuda()
            labels = labels.cuda()

            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(true_labels, predicted_labels)
    balanced_accuracy = balanced_accuracy_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')

    macro_f1 = f1_score(true_labels, predicted_labels, average='macro')
    macro_precision = precision_score(true_labels, predicted_labels, average='macro')
    macro_recall = recall_score(true_labels, predicted_labels, average='macro')

    print("Classification Report : \n", classification_report(true_labels, predicted_labels))

    return accuracy, balanced_accuracy, f1, precision, recall, macro_f1, macro_precision, macro_recall

In [ ]:

batch_size = 1024
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [ ]:
# Model Configuration
input_dim = X_train.shape[1]
num_classes = len(le.classes_)
model = TransformerModel(input_dim, num_classes)
model = model.cuda()

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Training loop
for epoch in range(5):
    model.train()
    running_loss = 0.0
    for i, (data, labels) in enumerate(train_loader):
        data = data.cuda()
        labels = labels.cuda()

        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print('Epoch [{}/10], Loss: {:.4f}'.format(epoch + 1, running_loss / len(train_loader)))

# Evaluation (replace with your evaluation function)
accuracy, balanced_accuracy, f1, precision, recall, macro_f1, macro_precision, macro_recall = evaluate_model(model, test_loader)
print('Macro F1 Score: {:.4f}'.format(macro_f1))
print('Macro Precision: {:.4f}'.format(macro_precision))
print('Macro Recall: {:.4f}'.format(macro_recall))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy))

In [ ]:
# Evaluation (replace with your evaluation function)
accuracy, balanced_accuracy, f1, precision, recall, macro_f1, macro_precision, macro_recall = evaluate_model(model, test_loader)
print('Macro F1 Score: {:.4f}'.format(macro_f1))
print('Macro Precision: {:.4f}'.format(macro_precision))
print('Macro Recall: {:.4f}'.format(macro_recall))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy))

Num heads: 10, d_model: 240, Epoch [1/10], Loss: 0.509706433875162


RuntimeError: ignored